# ETL OUALYON

In [27]:
import pandas as pd
import requests as rq
import json
import CategoryMap

In [28]:
dataframe = pd.DataFrame(columns=['Nom', 'Commune', 'Type','Coord_X', 'Coord_Y'])

#### Import de Sport Gouv

In [29]:
response = rq.get("https://equipements.sports.gouv.fr/api/records/1.0/search/?dataset=data-es&q=&refine.code_dept=69&rows=10000")
data = response.json()
for record in data['records']:
    rc = record['fields']
    dataframe.loc[len(dataframe.index)] = [rc['nominstallation'],rc['nom_commune'],rc['typequipement'],rc['coordgpsx'],rc['coordgpsy']]

#### Import de Data Grand Lyon

In [30]:
response = rq.get("https://download.data.grandlyon.com/ws/rdata/urbalyon.recenseqptsport/all.json?maxfeatures=10000&start=1")
data = response.json()
for record in data['values']: 
    dataframe.loc[len(dataframe.index)] = [record['nom'],record['commune'],record['type'],record['lon'],record['lat']]

In [31]:
# dataframe

#### Traitement des données

In [32]:
dataframe = dataframe.drop_duplicates() # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
dataframe

,Nom,Commune,Type,Coord_X,Coord_Y
0,Base de Loisirs,Aigueperse,Court de tennis,4.436730,46.277250
1,Salle des Fetes et Terrain de Jeu,Alix,Terrain de basket-ball,4.650480,45.914510
2,Collège Saint Viateur,Amplepuis,Terrain de basket-ball,4.330500,45.971400
3,Piscine Municipale,Amplepuis,Bassin sportif de natation,4.336410,45.979650
4,Salle de Gymnastique,Amplepuis,Salle de gymnastique sportive,4.312990,45.981010
...,...,...,...,...,...
8632,court de tennis municipaux 3,Givors,Court de tennis,4.770400,45.598930
8633,Terrain de football 11,Vénissieux,Terrain de football,4.896890,45.720730
8634,court de tennis municipaux 4,Givors,Court de tennis,4.770400,45.598930
8635,COURT DE TENNIS 2,Vénissieux,Court de tennis,4.894418,45.720817


Catégorisation des sports

In [33]:
category_map = CategoryMap.categories

dataframe['Category'] = dataframe['Type'].map(category_map)

In [34]:
#Traitements sur le dataframe
# dataframe.drop(dataframe[(dataframe.Commune != 'Amplepuis')].index,inplace=True)

#### Export en csv

In [35]:
dataframe.to_csv('activities.csv')

#### Export en GeoJSON

In [51]:
# Dico python vide :
GeoJson = {
  "type": "FeatureCollection",
  "features": []
}

In [52]:
# Alimentation des features
for index, row in dataframe.iterrows():
  feature = {
    "type": "Feature",
    "properties": {
      "Name": row["Nom"],
      "Commune" : row["Commune"],
      "Type" : row["Type"]
    },
    "geometry": {
      "coordinates": [
        row["Coord_X"],
        row["Coord_Y"]
      ],
      "type": "Point"
    }
  }
  
  GeoJson["features"].append(feature)


In [53]:
# convertion en JSON:
GeoJson = json.dumps(GeoJson)

In [54]:
# Création du fichier "GeoActivities.json"
with open("GeoActivities.json", "w") as outfile:
    outfile.write(GeoJson)